## Week 1: Discrete time Markov chain

* Simulating symmetric 1D random walk
* Simulating a Markov chain from its transition matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Example 1: Symmetric Random Walk

Initialization

In [ ]:
nsteps = 1000 #number of steps to simulate
x = np.zeros(nsteps)
x[0] = 0 #initial state

Simulation

In [ ]:
for t in range(nsteps - 1):
    if np.random.rand() <= 0.5:
        x[t + 1] = x[t] + 1 
    else:
        x[t + 1] = x[t] - 1

Plotting the outcome of the simulation

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(x, lw=2)

To simulate multiple realizations of the walk we just need to run the simulation above multiple times and store the results

Initialize each realization

In [ ]:
nrealizations = 10 #number of realizations to simulate

x = np.zeros((nsteps, nrealizations))
x[0] = np.zeros(nrealizations) #initial state

Simulate each realization

In [ ]:
for r in range(nrealizations):
    for t in range(nsteps - 1):
        if np.random.rand() <= 0.5:
            x[t + 1, r] = x[t, r] + 1 
        else:
            x[t + 1, r] = x[t, r] - 1

Plotting each realization

In [ ]:
for r in range(nrealizations):
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    ax.plot(x[:, r], lw=2)
    plt.show()

### Example 2: Simulating a Markov chain using the transition matrix

The following class allows to define a Markov Chain from a transition matrix and its states, and simulate the chain (from https://github.com/PacktPublishing/Hands-On-Markov-Models-with-Python)

In [ ]:
class MarkovChain(object):
    def __init__(self, transition_matrix, states):
        """
        Initialize the MarkovChain instance.
 
        Parameters
        ----------
        transition_matrix: 2-D array
            A 2-D array representing the probabilities of change of 
            state in the Markov Chain.
 
        states: 1-D array 
            An array representing the states of the Markov Chain. It
            needs to be in the same order as transition_matrix.
        """
        self.transition_matrix = np.atleast_2d(transition_matrix)
        self.states = states
        self.index_dict = {self.states[index]: index for index in 
                           range(len(self.states))}
        self.state_dict = {index: self.states[index] for index in
                           range(len(self.states))}
 
    def next_state(self, current_state):
        """
        Returns the state of the random variable at the next time 
        instance.
 
        Parameters
        ----------
        current_state: str
            The current state of the system.
        """
        return np.random.choice(
         self.states, 
         p=self.transition_matrix[self.index_dict[current_state], :]
        )
 
    def generate_states(self, current_state, no=10):
        """
        Generates the next states of the system.
 
        Parameters
        ----------
        current_state: str
            The state of the current random variable.
 
        no: int
            The number of future states to generate.
        """
        future_states = []
        for i in range(no):
            next_state = self.next_state(current_state)
            future_states.append(next_state)
            current_state = next_state
        return future_states

The next block allows to input a transition matrix, initialize the Markov chain and simulate a realization

In [ ]:
transition_matrix = [[0.5, 0.5], [0.5,  0.5]]# Input your transition matrix here
example_chain = MarkovChain(transition_matrix=transition_matrix, states=['0', '1']) # Name your states here
simulation= example_chain.generate_states(current_state='0', no=100) #Simulate the chain (specify the number of states here)
arr = np.array(simulation)
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(arr, lw=2)

To simulate multiple realizations of the chain we defined above, we simply call the `generate_states()` method multiple times

Initialize an array for storing all of our simulation results

In [ ]:
nrealizations = 10
nsteps = 100

arr = np.zeros((nsteps, nrealizations))

Run `generate_states()` once for each realization and store the result

In [ ]:
for r in range(nrealizations):
    simulation = example_chain.generate_states(current_state='0', no=nsteps)
    arr[:, r] = np.array(simulation)

Plot the realizations

In [ ]:
for r in range(nrealizations):
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    ax.plot(arr[:, r], lw=2)
    plt.show()

Compute the empirical distribution over the states at some time step `N` using the following function

In [ ]:
def get_empirical_dist(states, transition_matrix, initial_state, nrealizations, N):
    # Simulate the realizations
    chain = MarkovChain(transition_matrix=transition_matrix, states=states)
    arr = np.zeros(((N+1), nrealizations))
    
    for r in range(nrealizations):
        simulation = chain.generate_states(current_state=initial_state, no=(N+1))
        arr[:, r] = np.array(simulation)
        
    # Count the occurence of each state at time N
    nstates = len(states) #number of states
    count = np.zeros(nstates)
    
    for i in range(nstates):
        state = int(states[i])
        index = np.where(arr[N] == state)
        count[i] = len(index[0])
        
    # Compute the empirical distribution by normalizing by the total number of samples
    distribution = count/nrealizations
        
    return distribution

Here is an example showing the usage of this function

In [ ]:
empirical_distribution = get_empirical_dist(states=['0', '1'], 
                         transition_matrix=[[0.5, 0.5], [0.5,  0.5]], 
                         initial_state='0', 
                         nrealizations=100, 
                         N=10)

print(empirical_distribution)

We can plot this distribution as a simple histogram

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
bar_positions = np.arange(len(empirical_distribution))
ax.bar(bar_positions, empirical_distribution)
plt.xticks(bar_positions, ('0', '1'))
plt.show()

Exercises for students: See how to implement multiple examples, get empirical distributions and compare with theoretical results